<a href="https://www.kaggle.com/code/mohamedarish/with-dataset-as-input?scriptVersionId=175649209" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pyspark transformers torch tqdm scikit-learn sparknlp huggingface_hub fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 36.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=f8d7d71424aaba16fd539c45d07823144d520faaea3f8f4249f7c39489643548
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# LOGISTIC REGRESSION

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from huggingface_hub import hf_hub_download
import fasttext
import fasttext.util

model_path = hf_hub_download(repo_id="facebook/fasttext-ml-vectors", filename="model.bin")
model = fasttext.load_model(model_path)

# # Directory containing your CSV files
directory = '/kaggle/input/malayalam-tweets/'

# # List to store DataFrames from each CSV file
dfs = []

# # Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and "total" not in filename:
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        a_df = pd.read_csv(file_path)

        if "datetimee" in a_df.columns:
            # print("has datetimeee")
            a_df = a_df.rename(columns={"datetimee": "datetime"})
            # Append the DataFrame to the list
            dfs.append(a_df)

# # Combine all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

df = df.dropna(subset=['clean_content'])

df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})

# Step 3: Tokenization and Feature Extraction
def extract_features(text):
    tokens = text.split()  # Tokenize text into words
    vector_sum = sum(model.get_sentence_vector(token) for token in tokens)  # Get FastText vector for each word and sum them
    return vector_sum / len(tokens)  # Average the word vectors to get text vector

# Apply feature extraction to each text in the dataset
df['text_vector'] = df['clean_content'].apply(extract_features)

# Step 4: Split Data
X_train, X_test, y_train, y_test = train_test_split(df['text_vector'], df['sentiment'], test_size=0.2, random_state=42)

# Step 5: Initialize Logistic Regression Model
logistic_regression_model = LogisticRegression(max_iter=1000)

# Step 6: Train Model in Epochs
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    tqdm_bar = tqdm(total=len(X_train), position=0, leave=True)  # Create tqdm progress bar
    
    # Train the model
    logistic_regression_model.fit(X_train.tolist(), y_train)
    
    # Calculate training metrics
    train_loss = None  # Logistic regression does not have a loss attribute
    train_accuracy = accuracy_score(y_train, logistic_regression_model.predict(X_train.tolist()))
    train_precision = precision_score(y_train, logistic_regression_model.predict(X_train.tolist()), average='weighted')
    train_recall = recall_score(y_train, logistic_regression_model.predict(X_train.tolist()), average='weighted')
    train_f1_score = f1_score(y_train, logistic_regression_model.predict(X_train.tolist()), average='weighted')
    
    # Calculate validation metrics
    val_accuracy = accuracy_score(y_test, logistic_regression_model.predict(X_test.tolist()))
    val_precision = precision_score(y_test, logistic_regression_model.predict(X_test.tolist()), average='weighted')
    val_recall = recall_score(y_test, logistic_regression_model.predict(X_test.tolist()), average='weighted')
    val_f1_score = f1_score(y_test, logistic_regression_model.predict(X_test.tolist()), average='weighted')
    
    tqdm_bar.close()  # Close tqdm progress bar
    
    # Print metrics
    print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1-score: {train_f1_score}")
    print(f"Validation Accuracy: {val_accuracy}, Validation Precision: {val_precision}, Validation Recall: {val_recall}, Validation F1-score: {val_f1_score}")


Epoch 1/1


  0%|          | 0/7171 [00:00<?, ?it/s]

Train Loss: None, Train Accuracy: 0.6739645795565472, Train Precision: 0.6817086427052726, Train Recall: 0.6739645795565472, Train F1-score: 0.6576976785981656
Validation Accuracy: 0.637479085331846, Validation Precision: 0.6396109859134467, Validation Recall: 0.637479085331846, Validation F1-score: 0.617633474977311


# RANDOM FOREST

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import fasttext.util

model_path = hf_hub_download(repo_id="facebook/fasttext-ml-vectors", filename="model.bin")
model = fasttext.load_model(model_path)

# # Directory containing your CSV files
directory = '/kaggle/input/malayalam-tweets/'

# # List to store DataFrames from each CSV file
dfs = []

# # Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and "total" not in filename:
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        a_df = pd.read_csv(file_path)

        if "datetimee" in a_df.columns:
            # print("has datetimeee")
            a_df = a_df.rename(columns={"datetimee": "datetime"})
            # Append the DataFrame to the list
            dfs.append(a_df)

# # Combine all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

df = df.dropna(subset=['clean_content'])

df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})

# Step 3: Tokenization and Feature Extraction
def extract_features(text):
    tokens = text.split()  # Tokenize text into words
    vector_sum = sum(model.get_sentence_vector(token) for token in tokens)  # Get FastText vector for each word and sum them
    return vector_sum / len(tokens)  # Average the word vectors to get text vector

# Apply feature extraction to each text in the dataset
df['text_vector'] = df['clean_content'].apply(extract_features)

# Step 4: Split Data
X_train, X_test, y_train, y_test = train_test_split(df['text_vector'], df['sentiment'], test_size=0.2, random_state=42)

# Step 5: Initialize Logistic Regression Model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 6: Train Model in Epochs
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    tqdm_bar = tqdm(total=len(X_train), position=0, leave=True)  # Create tqdm progress bar
    
    # Train the model
    random_forest_model.fit(X_train.tolist(), y_train)
    
    # Calculate training metrics
    train_loss = None  # Logistic regression does not have a loss attribute
    train_accuracy = accuracy_score(y_train, random_forest_model.predict(X_train.tolist()))
    train_precision = precision_score(y_train, random_forest_model.predict(X_train.tolist()), average='weighted')
    train_recall = recall_score(y_train, random_forest_model.predict(X_train.tolist()), average='weighted')
    train_f1_score = f1_score(y_train, random_forest_model.predict(X_train.tolist()), average='weighted')
    
    # Calculate validation metrics
    val_accuracy = accuracy_score(y_test, random_forest_model.predict(X_test.tolist()))
    val_precision = precision_score(y_test, random_forest_model.predict(X_test.tolist()), average='weighted')
    val_recall = recall_score(y_test, random_forest_model.predict(X_test.tolist()), average='weighted')
    val_f1_score = f1_score(y_test, random_forest_model.predict(X_test.tolist()), average='weighted')
    
    tqdm_bar.close()  # Close tqdm progress bar
    
    # Print metrics
    print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1-score: {train_f1_score}")
    print(f"Validation Accuracy: {val_accuracy}, Validation Precision: {val_precision}, Validation Recall: {val_recall}, Validation F1-score: {val_f1_score}")


Epoch 1/1


  0%|          | 0/7171 [00:11<?, ?it/s]

Train Loss: None, Train Accuracy: 0.9995816483056756, Train Precision: 0.9995819474761473, Train Recall: 0.9995816483056756, Train F1-score: 0.999581616854672
Validation Accuracy: 0.6296709425543782, Validation Precision: 0.6450759085951593, Validation Recall: 0.6296709425543782, Validation F1-score: 0.5991409452797823


# SVM

In [6]:
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import fasttext

model_path = hf_hub_download(repo_id="facebook/fasttext-ml-vectors", filename="model.bin")
model = fasttext.load_model(model_path)

# # Directory containing your CSV files
directory = '/kaggle/input/malayalam-tweets/'

# # List to store DataFrames from each CSV file
dfs = []

# # Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and "total" not in filename:
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        a_df = pd.read_csv(file_path)

        if "datetimee" in a_df.columns:
            # print("has datetimeee")
            a_df = a_df.rename(columns={"datetimee": "datetime"})
            # Append the DataFrame to the list
            dfs.append(a_df)

# # Combine all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

df = df.dropna(subset=['clean_content'])

df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})

# Step 3: Tokenization and Feature Extraction
def extract_features(text):
    tokens = text.split()  # Tokenize text into words
    vector_sum = sum(model.get_sentence_vector(token) for token in tokens)  # Get FastText vector for each word and sum them
    return vector_sum / len(tokens)  # Average the word vectors to get text vector

# Apply feature extraction to each text in the dataset
df['text_vector'] = df['clean_content'].apply(extract_features)

# Step 4: Split Data
X_train, X_test, y_train, y_test = train_test_split(df['text_vector'], df['sentiment'], test_size=0.2, random_state=42)

# Step 5: Initialize Logistic Regression Model
svm_model = SVC(kernel='linear', random_state=42)

# Step 6: Train Model in Epochs
epochs = 1
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    tqdm_bar = tqdm(total=len(X_train), position=0, leave=True)  # Create tqdm progress bar
    
    # Train the model
    svm_model.fit(X_train.tolist(), y_train)
    
    # Calculate training metrics
    train_loss = None  # Logistic regression does not have a loss attribute
    train_accuracy = accuracy_score(y_train, svm_model.predict(X_train.tolist()))
    train_precision = precision_score(y_train, svm_model.predict(X_train.tolist()), average='weighted')
    train_recall = recall_score(y_train, svm_model.predict(X_train.tolist()), average='weighted')
    train_f1_score = f1_score(y_train, svm_model.predict(X_train.tolist()), average='weighted')
    
    # Calculate validation metrics
    val_accuracy = accuracy_score(y_test, svm_model.predict(X_test.tolist()))
    val_precision = precision_score(y_test, svm_model.predict(X_test.tolist()), average='weighted')
    val_recall = recall_score(y_test, svm_model.predict(X_test.tolist()), average='weighted')
    val_f1_score = f1_score(y_test, svm_model.predict(X_test.tolist()), average='weighted')
    
    tqdm_bar.close()  # Close tqdm progress bar
    
    # Print metrics
    print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1-score: {train_f1_score}")
    print(f"Validation Accuracy: {val_accuracy}, Validation Precision: {val_precision}, Validation Recall: {val_recall}, Validation F1-score: {val_f1_score}")


Epoch 1/1


  0%|          | 0/7171 [00:31<?, ?it/s]

Train Loss: None, Train Accuracy: 0.6584855668665458, Train Precision: 0.6789439172896629, Train Recall: 0.6584855668665458, Train F1-score: 0.6297087537970312
Validation Accuracy: 0.6291132180702733, Validation Precision: 0.6349227133713751, Validation Recall: 0.6291132180702733, Validation F1-score: 0.5983744133620498


# BIDIRECTIONAL LSTM WITH ATTENTION LAYER

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Concatenate, Activation, Lambda, RepeatVector, Permute, Flatten
import tensorflow.keras.backend as K
import os
import torch

def clear_gpu_memory():
  """Frees memory allocated on the GPU."""
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

# # Directory containing your CSV files
directory = '/kaggle/input/malayalam-tweets/'

# # List to store DataFrames from each CSV file
dfs = []

# # Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        a_df = pd.read_csv(file_path)

        if "datetimee" in a_df.columns:
            # print("has datetimeee")
            a_df = a_df.rename(columns={"datetimee": "datetime"})
            # Append the DataFrame to the list
        dfs.append(a_df)

# # Combine all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# print(df.size)

df = df.dropna(subset=['clean_content'])

print(df.size)
df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})

# Preprocessing
maxlen = 10000  # Maximum sequence length
max_words = 200000  # Maximum number of words in vocabulary

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(df['content'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['content'])

# Pad sequences to maxlen
X = pad_sequences(sequences, maxlen=maxlen)

# Label encoding for sentiments (assuming you have 'sentiment' column)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['sentiment'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Bidirectional LSTM with Attention Model
input_seq = Input(shape=(maxlen,))
embedding = Embedding(max_words, 128, input_length=maxlen)(input_seq)
lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding)

# Attention Mechanism
attention = Dense(1, activation='tanh')(lstm)
attention = Flatten()(attention)
attention = Activation('softmax', name='attention_weights')(attention)
attention = RepeatVector(128 * 2)(attention)
attention = Permute([2, 1])(attention)

sent_representation = Concatenate(axis=-1)([lstm, attention])
sent_representation = Lambda(lambda x: K.sum(x, axis=1))(sent_representation)

output = Dense(1, activation='sigmoid')(sent_representation)

model = Model(inputs=input_seq, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

clear_gpu_memory()

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

# Print training and validation metrics
print("Training Loss:", history.history['loss'])
print("Training Accuracy:", history.history['accuracy'])
print("Validation Loss:", history.history['val_loss'])
print("Validation Accuracy:", history.history['val_accuracy'])

64680
64659


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 124s 573ms/step - accuracy: 0.4114 - loss: -25902.7051 - val_accuracy: 0.4154 - val_loss: -68951.6797
Epoch 2/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 120s 576ms/step - accuracy: 0.4128 - loss: -81250.4688 - val_accuracy: 0.4154 - val_loss: -120757.0312
Epoch 3/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 120s 578ms/step - accuracy: 0.4089 - loss: -157105.2656 - val_accuracy: 0.4154 - val_loss: -192876.8438
Epoch 4/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 120s 577ms/step - accuracy: 0.4001 - loss: -311555.7812 - val_accuracy: 0.4154 - val_loss: -268831.4688
Epoch 5/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 120s 577ms/step - accuracy: 0.4020 - loss: -418399.0312 - val_accuracy: 0.4154 - val_loss: -341662.7812
Epoch 6/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 120s 577ms/step - accuracy: 0.4063 - loss: -512298.0938 - val_accuracy: 0.4154 - val_loss: -398410.5938
Epoch 7/10
208/208 ━━━━━━━━━━━━━━━━━━━━ 120s 577ms/step - accuracy: 0.4089 - loss: -618858.2500 - val_accuracy: 0.4154 - val_loss: -481179.

# BERT

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.optim import AdamW
from torch.optim.lr_scheduler import ExponentialLR
import os
import pandas as pd
import torch

def clear_gpu_memory():
  """Frees memory allocated on the GPU."""
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Load Malayalam BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('l3cube-pune/malayalam-bert')

def create_malayalam_dataset(content_col, sentiment_col, max_len=128):
    tokenizer = BertTokenizer.from_pretrained('l3cube-pune/malayalam-bert')

    def tokenize_text(content, sentiment):
        inputs = tokenizer.encode_plus(
            content,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': content,
            'input_ids': inputs['input_ids'].flatten().tolist(),
            'attention_mask': inputs['attention_mask'].flatten().tolist(),
            'target': int(sentiment)
        }

    return udf(tokenize_text, StructType([
        StructField('text', StringType(), True),
        StructField('input_ids', ArrayType(IntegerType()), True),
        StructField('attention_mask', ArrayType(IntegerType()), True),
        StructField('target', IntegerType(), True)
    ]))


# Define dataset class
class MalayalamDataset(Dataset):
    def __init__(self, dataframe, max_len=128):
        self.data = dataframe
        self.max_len = max_len
        self.texts = self.data.content.tolist()
        self.targets = self.data.sentiment.tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        target = self.targets[index]

        inputs = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'target': torch.tensor(target, dtype=torch.long)
        }

# Load and preprocess dataset
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    return df

# Fine-tune BERT for sentiment analysis
def train_model(train_loader, val_loader, model, optimizer, scheduler, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total_targets = 0
        for batch in tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['target'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            total_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
            correct += (preds == targets.cpu().numpy()).sum()
            total_targets += len(targets)

            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = correct / total_targets
        print(f"Average training loss for epoch {epoch+1}: {avg_train_loss}")
        print(f"Training Accuracy for epoch {epoch+1}: {train_accuracy}")

        val_loss, val_acc, val_report = evaluate_model(model, val_loader, device)
        print(f"Validation loss: {val_loss}, Accuracy: {val_acc}")
        print("Validation Report:")
        print(val_report)

        scheduler.step()

# Evaluate the model
def evaluate_model(model, val_loader, device):
    model.eval()
    val_targets = []
    val_outputs = []

    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['target'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).detach().cpu().numpy()

            val_targets.extend(targets.cpu().numpy())
            val_outputs.extend(preds)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_targets, val_outputs)
    val_report = classification_report(val_targets, val_outputs, target_names=["Negative", "Neutral", "Positive"])
    return avg_val_loss, val_accuracy, val_report
# Main function to train and evaluate
def main():
  tokenizer.save_pretrained("bert-tokenizer")

# Create a SparkSession
    spark = SparkSession.builder \
        .appName("Malayalam Tweets Analysis") \
        .getOrCreate()

    # Read the CSV files into a Spark DataFrame
    directory = '/kaggle/input/malayalam-tweets/'

    # # List to store DataFrames from each CSV file
    dfs = []

    # # Loop through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv") and "total" not in filename:
          file_path = os.path.join(directory, filename)
          # Read the CSV file into a DataFrame
          a_df = pd.read_csv(file_path)

          if "datetimee" in a_df.columns:
            # print("has datetimeee")
            a_df = a_df.rename(columns={"datetimee": "datetime"})
            # Append the DataFrame to the list
        dfs.append(a_df)

    # # Combine all DataFrames into a single DataFrame
    df = pd.concat(dfs, ignore_index=True)

    df = df.dropna(subset=['clean_content'])

    df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})
    
    df_spark = spark.createDataFrame(df)

    # Define UDF to create Malayalam Dataset
    create_dataset_udf = create_malayalam_dataset("clean_content", "sentiment")

    # Apply UDF to create train_dataset and val_dataset
    df_spark = df_spark.withColumn("malayalam_dataset", create_dataset_udf(df_spark["clean_content"], df_spark["sentiment"]))

    # Split dataset into train and validation
    train_df, val_df = df_spark.randomSplit([0.9, 0.1], seed=42)

    # Convert Spark DataFrame to pandas DataFrame (optional)
    train_df_pandas = train_df.select("malayalam_dataset").toPandas()
    val_df_pandas = val_df.select("malayalam_dataset").toPandas()

    # Convert pandas DataFrames to datasets
    train_dataset = [row.malayalam_dataset for _, row in train_df_pandas.iterrows()]
    val_dataset = [row.malayalam_dataset for _, row in val_df_pandas.iterrows()]
    
    spark.stop()

#   return
  # # Directory containing your CSV files
  directory = '/kaggle/input/malayalam-tweets/'

  # # List to store DataFrames from each CSV file
  dfs = []

  # # Loop through each file in the directory
  for filename in os.listdir(directory):
    if filename.endswith(".csv") and "total" not in filename:
      file_path = os.path.join(directory, filename)
      # Read the CSV file into a DataFrame
      a_df = pd.read_csv(file_path)

      if "datetimee" in a_df.columns:
        # print("has datetimeee")
        a_df = a_df.rename(columns={"datetimee": "datetime"})
      # Append the DataFrame to the list
      dfs.append(a_df)

  # # Combine all DataFrames into a single DataFrame
  df = pd.concat(dfs, ignore_index=True)

  df = df.dropna(subset=['clean_content'])

  df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})

  # Split dataset into train and validation
  train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

  # Create datasets and dataloaders
  train_dataset = MalayalamDataset(train_df)
  val_dataset = MalayalamDataset(val_df)

  batch_size = 16
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

  # Load pre-trained BERT model for sequence classification
  model = BertForSequenceClassification.from_pretrained('l3cube-pune/malayalam-bert', num_labels=3)

  # Send model to GPU, if available
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  # Create optimizer
  optimizer = AdamW(model.parameters(), lr=2e-5)

  # Create scheduler
  scheduler = ExponentialLR(optimizer, gamma=0.9)

  # Train the model
  train_model(train_loader, val_loader, model, optimizer, scheduler, device, epochs=9)

  clear_gpu_memory()

  # Save the trained model
  model.save_pretrained("malayalam_sentiment_model")

if __name__ == "__main__":
  main()

tokenizer_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

...

Average training loss for epoch 7: 0.05269081321938966
Training Accuracy for epoch 7: 0.9892938770600265
Validation loss: 0.901808185567116, Accuracy: 0.7662337662337663
Validation Report:
              precision    recall  f1-score   support

    Negative       0.70      0.76      0.73       149
     Neutral       0.73      0.75      0.74       397
    Positive       0.84      0.78      0.81       378

    accuracy                           0.77       924
   macro avg       0.76      0.76      0.76       924
weighted avg       0.77      0.77      0.77       924



Epoch 8/9: 100%|██████████| 520/520 [03:16<00:00,  2.65it/s]


Average training loss for epoch 8: 0.04330157337961002
Training Accuracy for epoch 8: 0.9904968122218213
Validation loss: 0.9412522490682274, Accuracy: 0.7673160173160173
Validation Report:
              precision    recall  f1-score   support

    Negative       0.71      0.69      0.70       149
     Neutral       0.74      0.72      0.73       397
    Positive       0.81      0.84      0.83       378

    accuracy                           0.77       924
   macro avg       0.76      0.75      0.75       924
weighted avg       0.77      0.77      0.77       924



Epoch 9/9: 100%|██████████| 520/520 [03:16<00:00,  2.65it/s]


Average training loss for epoch 9: 0.0351567970123142
Training Accuracy for epoch 9: 0.9932635630939493
Validation loss: 0.9613229227734024, Accuracy: 0.762987012987013
Validation Report:
              precision    recall  f1-score   support

    Negative       0.72      0.65      0.68       149
     Neutral       0.73      0.75      0.74       397
    Positive       0.81      0.82      0.82       378

    accuracy                           0.76       924
   macro avg       0.75      0.74      0.75       924
weighted avg       0.76      0.76      0.76       924



# BERT WITH SPARK

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
import os
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from torch.optim import AdamW
from torch.optim.lr_scheduler import ExponentialLR
import os
import pandas as pd
import torch

def clear_gpu_memory():
  """Frees memory allocated on the GPU."""
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

def create_malayalam_dataset(content_col, sentiment_col, max_len=128):
    tokenizer = BertTokenizer.from_pretrained('l3cube-pune/malayalam-bert')

    def tokenize_text(content, sentiment):
        inputs = tokenizer.encode_plus(
            content,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': content,
            'input_ids': inputs['input_ids'].flatten().tolist(),
            'attention_mask': inputs['attention_mask'].flatten().tolist(),
            'target': int(sentiment)
        }

    return udf(tokenize_text, StructType([
        StructField('text', StringType(), True),
        StructField('input_ids', ArrayType(IntegerType()), True),
        StructField('attention_mask', ArrayType(IntegerType()), True),
        StructField('target', IntegerType(), True)
    ]))


# Fine-tune BERT for sentiment analysis
def train_model(train_loader, val_loader, model, optimizer, scheduler, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total_targets = 0
        for batch in tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}"):

#             input_ids = batch['input_ids'].to(device)
            input_ids = batch[1]
#             attention_mask = batch['attention_mask'].to(device)
            attention_mask = batch[2]
#             targets = batch['target'].to(device)
            targets = batch[3]

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            total_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).detach().cpu().numpy()
            correct += (preds == targets.cpu().numpy()).sum()
            total_targets += len(targets)

            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = correct / total_targets
        print(f"Average training loss for epoch {epoch+1}: {avg_train_loss}")
        print(f"Training Accuracy for epoch {epoch+1}: {train_accuracy}")

        val_loss, val_acc, val_report = evaluate_model(model, val_loader, device)
        print(f"Validation loss: {val_loss}, Accuracy: {val_acc}")
        print("Validation Report:")
        print(val_report)

        scheduler.step()

# Evaluate the model
def evaluate_model(model, val_loader, device):
    model.eval()
    val_targets = []
    val_outputs = []

    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
#             input_ids = batch['input_ids'].to(device)
            input_ids = batch[1]
#             attention_mask = batch['attention_mask'].to(device)
            attention_mask = batch[2]
#             targets = batch['target'].to(device)
            targets = batch[3]

            outputs = model(input_ids, attention_mask=attention_mask, labels=targets)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).detach().cpu().numpy()

            val_targets.extend(targets.cpu().numpy())
            val_outputs.extend(preds)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_targets, val_outputs)
    val_report = classification_report(val_targets, val_outputs, target_names=["Negative", "Neutral", "Positive"])
    return avg_val_loss, val_accuracy, val_report

def main():
    # Create a SparkSession
    spark = SparkSession.builder \
        .appName("Malayalam Tweets Analysis") \
        .getOrCreate()

    # Read the CSV files into a Spark DataFrame
    directory = '/kaggle/input/malayalam-tweets/'

    # # List to store DataFrames from each CSV file
    dfs = []

    # # Loop through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv") and "total" not in filename:
          file_path = os.path.join(directory, filename)
          # Read the CSV file into a DataFrame
          a_df = pd.read_csv(file_path)

          if "datetimee" in a_df.columns:
            # print("has datetimeee")
            a_df = a_df.rename(columns={"datetimee": "datetime"})
            # Append the DataFrame to the list
        dfs.append(a_df)

    # # Combine all DataFrames into a single DataFrame
    df = pd.concat(dfs, ignore_index=True)

    df = df.dropna(subset=['clean_content'])

    df['sentiment'] = df['sentiment'].map({-1: 0, 0: 1, 1: 2})
    
    df_spark = spark.createDataFrame(df)

    # Define UDF to create Malayalam Dataset
    create_dataset_udf = create_malayalam_dataset("clean_content", "sentiment")

    # Apply UDF to create train_dataset and val_dataset
    df_spark = df_spark.withColumn("malayalam_dataset", create_dataset_udf(df_spark["clean_content"], df_spark["sentiment"]))

    # Split dataset into train and validation
    train_df, val_df = df_spark.randomSplit([0.9, 0.1], seed=42)

    # Convert Spark DataFrame to pandas DataFrame (optional)
    train_df_pandas = train_df.select("malayalam_dataset").toPandas()
    val_df_pandas = val_df.select("malayalam_dataset").toPandas()

    # Convert pandas DataFrames to datasets
    train_dataset = [row.malayalam_dataset for _, row in train_df_pandas.iterrows()]
    val_dataset = [row.malayalam_dataset for _, row in val_df_pandas.iterrows()]
    # Other steps (not included in this snippet)
    # Initialize model, optimizer, scheduler
    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=batch_size)
    # train_model(train_loader, val_loader, model, optimizer, scheduler, device, epochs=10)
    
    print("Split done")
    
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    print("Loaders loaded")

    # Load pre-trained BERT model for sequence classification
    model = BertForSequenceClassification.from_pretrained('l3cube-pune/malayalam-bert', num_labels=3)
    
    print("Model loaded")

    # Send model to GPU, if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    print("GPU done")

    # Create optimizer
    optimizer = AdamW(model.parameters(), lr=2e-5)
    
    print("Optimizer loaded")

    # Create scheduler
    scheduler = ExponentialLR(optimizer, gamma=0.9)
    
    print("Scheduler loaded")

    # Train the model
    train_model(train_loader, val_loader, model, optimizer, scheduler, device, epochs=9)
    
    print("Model trained")

    clear_gpu_memory()
    
    print("gpu memory cleared")
    
if __name__ == "__main__":
  main()

24/05/04 15:40:40 WARN TaskSetManager: Stage 10 contains a task of very large size (1951 KiB). The maximum recommended task size is 1000 KiB.
24/05/04 15:40:46 WARN TaskSetManager: Stage 11 contains a task of very large size (1951 KiB). The maximum recommended task size is 1000 KiB.


Split done
Loaders loaded


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/malayalam-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded
GPU done
Optimizer loaded
Scheduler loaded


Epoch 1/9:   0%|          | 0/524 [00:00<?, ?it/s]

[('ആർക്ക് സാക്ഷരത  ഇസ്ലാമിക തീവ്രവാദികളുടെ ആസനം  താങ്ങുന്ന ഇടത് വലത് രാഷ്രീയ നപുസ്ഥകങ്ങൾക്കോ  തീവ്രവാദിയെ പിടിക്കാൻ ഡൽഹിയിൽ  നിന്നും  പ്രതേക വ്യാമസേനാ വിമാനം ഇറക്കിയ സംസ്ഥാന മാണ്', 'നമുക്ക് ഇഷ്ടപെട്ട ആളെ കെട്ടിയില്ലെങ്കിൽ കെട്ടുന്ന പെണ്ണിനെ കഷ്ട പെട്ടു ഇഷ്ടപെടേണ്ടിവരും by മൂവി', '2019 ല്\u200d വീണ്ടും അധികാരത്തിലേറിയ ശേഷമാണ് മോദി സര്\u200dക്കാര്\u200d പരിസ്ഥിതി ആഘാത വിലയിരുത്തലില്\u200d ഭേദഗതിക്കായി കരടു തയ്യാറാക്കിയത്.യുപിഎ സര്\u200dക്കാരിന്റെ കാലത്ത് അന്ന് പരിസ്ഥിതി മന്ത്രിയായിരുന്ന ജയറാം രമേശ് തയ്യാറാക്കിയിരുന്ന പരിസ്ഥിതി കരടു ഭേദഗതികള്\u200d പരിസ്ഥിതി മൗലികവാദത്തില്\u200d', 'സേവാഭാരതിയുടെ ഒരാഴ്ച്ചത്തെ കഠിനാദ്ധ്വാനം നാടിന് സമര്\u200dപ്പിച്ചു; കോമളം കടവിലെ പാലം ഇരുകരതൊട്ടു; അഭിനന്ദന പ്രവാഹവുമായി നാട്ടുകാര്\u200d', 'ഇത് പേടി അല്ല... ജാഗ്രത മാത്രം...   ബാലൻ സാറിന് അങ്ങനെ ഒക്കെ പറയാം...  ബാലൻ സാറിന് വേർഡിനെ പറ്റി, എക്സലിനെ പറ്റി, എക്സലോജിക്കിനെ പറ്റി എന്ത് അറിയാം..  പിടിക്കപ്പെട്ടാൽ പെടുന്നതും വീണ് പോകുന്നതും വീണ അല്ല്യോ ബാലൻ സാറേ...  ഒക്കെ പെൻഷൻ കാശ് ആണ് - ആകെ ഉള്ള ഒരു സമാധാനം', 'ഒരു ച

AttributeError: 'list' object has no attribute 'size'